In [36]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def northwest_corner_method(supply, demand, costs):
    result_matrix = np.zeros((len(supply), len(demand)))

    i, j = 0, 0

    while i < len(supply) and j < len(demand):
        quantity = min(supply[i], demand[j])

        result_matrix[i, j] = quantity

        supply[i] -= quantity
        demand[j] -= quantity

        if supply[i] == 0:
            i += 1
        else:
            j += 1

    return result_matrix

def calculate_total_cost(result_matrix, costs):
    total_cost = np.sum(result_matrix * costs)
    return total_cost


supply1 = [310,260,280,360,220] 
demand1 = [170,	200,	180,	210,	240,	180,	150] 
costs1 = np.array([[10,	20,	18,	8,	16,	5,	8,],
[21,	14,	5,	20,	17,	15,	12,],
[17,	11,	21,	12,	21,	18,	17,],
[17,	15,	9,	18,	16,	19,	18,],
[11,	16,	20,	17,	6,	9,	14,]])

result_matrix1 = northwest_corner_method(supply1, demand1, costs1)
print("Матриця розподілу:")
print(result_matrix1)

total_cost1 = calculate_total_cost(result_matrix1, costs1)
print("Загальна вартість перевезень:", total_cost1)

Матриця розподілу:
[[170. 140.   0.   0.   0.   0.   0.]
 [  0.  60. 180.  20.   0.   0.   0.]
 [  0.   0.   0. 190.  90.   0.   0.]
 [  0.   0.   0.   0. 150. 180.  30.]
 [  0.   0.   0.   0.   0.   0. 120.]]
Загальна вартість перевезень: 18850.0


In [37]:
from pulp import *
import pandas as pd


supply2 = [200,350,300]
demand2 = [270,	130,	190,	150,	110,] 
costs2 = np.array([[24,	50,	5,	27,	16],
                    [50,	47,	23,	17,	21],
                    [35,	59,	55,	27,	41]])

parameters = makeDict([supply2, demand2], costs2, 0)
prob = LpProblem("Transportation_Problem", LpMinimize)

routes = [(i, j) for i in range(len(supply2)) for j in range(len(demand2))]

vars = LpVariable.dicts("Route", (range(len(supply2)), range(len(demand2))), 0, None, LpInteger)

prob += (
    lpSum([vars[w][b] * costs2[w][b] for (w, b) in routes]),
    "Sum_of_Transporting_Costs"
)

for s in range(len(supply2)):
    prob += (
        lpSum([vars[s][b] for b in range(len(demand2))]) <= supply2[s],
        "Sum_of_Products_out_of_supplier_%s" % s,
    )

for c in range(len(demand2)):
    prob += (
        lpSum([vars[w][c] for w in range(len(supply2))]) >= demand2[c],
        "Sum_of_Products_into_consumer%s" % c,
    )

prob.solve()

results = [v.varValue for v in prob.variables()]

print("Відповідь:")
print(pd.DataFrame(np.array(results).reshape(3,5)))
print("Загальна вартість:", value(prob.objective))


Відповідь:
       0      1      2      3      4
0    0.0    0.0  190.0    0.0   10.0
1    0.0  130.0    0.0  120.0  100.0
2  270.0    0.0    0.0   30.0    0.0
Загальна вартість: 21620.0
